# Importación de Liberias

In [2]:
#Para Modelado y limpieza de datos
import pandas as pd
import numpy as np
#Para graficar
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor



from sklearn.metrics import mean_squared_error, classification_report,confusion_matrix, roc_auc_score, roc_curve


# Lectura de archivo

In [8]:
url = "https://raw.githubusercontent.com/tommad1/NeuroBox/NeuroBox/BootCamp/Modulo%204/Tarea/marketing_campaign.csv"
data = pd.read_csv(url, sep=';')
#https://vscode.dev/github/tommad1/NeuroBox/blob/NeuroBox/BootCamp/Modulo%204/Tarea/marketing_campaign.csv
marketing

HTTPError: HTTP Error 404: Not Found